In [0]:
%pip install -U langchain-databricks langchain-core langchain mlflow

dbutils.library.restartPython()

In [0]:
%restart_python

## Imports

In [0]:
import mlflow
from langchain_databricks import ChatDatabricks
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

## Creating Dummy data

In [0]:
%sql
-- Create catalog if it doesn't exist
CREATE CATALOG IF NOT EXISTS main;

-- Switch to the catalog
USE CATALOG main;

-- Create schema if it doesn't exist
CREATE SCHEMA IF NOT EXISTS agents_demo;

-- Switch to the schema
USE SCHEMA agents_demo;

-- Create the table
CREATE OR REPLACE TABLE user_data (
  user_id INT,
  name STRING,
  account_balance DECIMAL(10, 2)
);

-- Insert data
INSERT INTO user_data VALUES
  (1, 'Alice', 1500.75),
  (2, 'Bob', 200.00),
  (3, 'Charlie', 5000.50);

## Delta Query tool

In [0]:
@tool
def get_user_balance(user_id: int) -> str:
  """
  Finds the account balance for a specific user_id from the user_data table.
  """
  print(f"--- Tool Called: get_user_balance(user_id={user_id}) ---")
  
  # The Delta Query
  try:
    df = spark.sql(f"""
      SELECT account_balance 
      FROM main.agents_demo.user_data 
      WHERE user_id = {user_id}
    """)
    
    result = df.first()
    
    if result:
      return f"The balance for user_id {user_id} is ${result['account_balance']}"
    else:
      return f"No user found with user_id {user_id}"
      
  except Exception as e:
    return f"Error querying database: {str(e)}"

# Put our tool into a list for the agent
tools = [get_user_balance]

## Define and Configure the Agent

In [0]:
llm = ChatDatabricks(
    endpoint="databricks-gpt-oss-120b",
    max_tokens=200
)

In [0]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. You must use your tools to answer questions."),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}"), # This is where the agent thinks
    ]
)

## Create the agent

In [0]:
# Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor (the runtime for the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [0]:
# 1. Set the MLflow experiment where logs will go
mlflow.set_experiment("/Users/ebrahemelsherif666i@gmail.com/minimal_agent_demo")

# 2. Enable MLflow autologging for LangChain
mlflow.langchain.autolog(log_models=True, log_input_examples=True)

# 3. Start your MLflow run
with mlflow.start_run(run_name="Minimal Agent Run") as run:
  
  # --- Our "One MLflow Log" ---
  # We explicitly log a key parameter for this run
  mlflow.log_param("agent_model", "databricks-dbrx-instruct")
  
  # --- Run the Agent ---
  print("--- Invoking Agent ---")
  user_question = "Hi, can you please tell me the balance for user_id 2?"
  
  response = agent_executor.invoke({
      "input": user_question,
  })

print("\n--- Agent's Final Answer ---")
print(response["output"])